# Saddle Point Locator
---

[鞍点](https://zh.m.wikipedia.org/zh-hans/鞍點)是一种特殊的驻点。对于多变量函数，在鞍点位置，函数沿任意方向的导数都为0，但函数并不是最大值或者最小值。我们关注一类特殊的鞍点，在这个位置，函数在某一方向上是最大值，但是在剩余所有方向上是极小值。

寻找鞍点在科学和工程研究中有很多应用。一个常用的例子是地形图，地势高度取决于水平坐标，因此这是一个双变量函数。假设在起伏的地势中有两个盆地（对应于函数的局部极小值）A和B。一个人想要从A出发到达B，在连接A和B的所有可能的路径中，哪一条路径走过的地势最高点最低？这个问题的实质就是寻找这个双变量函数的鞍点（或者一个更常见的名称，过渡态）。

## 作业题目
---

考虑一个三变量函数（见下方代码），寻找这个函数的在(0.5, 0.5, 0.5)和(-0.5, -0.5, -0.5)附近的两个局部极小值，以及两个极小值之间路径上(0.7, 0.3, 0.5)附近的鞍点。

**要求**
1. 数值结果误差小于1e-3。
2. 不允许使用numpy和scipy之外的数学库，需手动实现算法。
3. 不允许对提供的data的生成函数做符号分析（例如解析求导），data只能作为一个黑盒子使用。
4. 为了模拟真实场景（data中数据点需要借助实验或者模拟仿真高成本获得），不允许对data进行遍历或暴力搜索。
5. 代码优越性的评价取决于data中数据的读取次数，越低越好（对于次数越低，对应于真实场景中解决问题速度越快）。

为了方便大家对函数值的分布有直观认识，这里使用ipyvolume进行了3D可视化。

In [10]:
import numpy as np
import ipyvolume as ipv

x = np.linspace(-1, 1, 100)
y = np.linspace(-1, 1, 100)
z = np.linspace(-1, 1, 100)
Y, Z, X = np.meshgrid(x, y, z)

def gaussian(x0, y0, z0):
    return np.exp(-(X - x0)**2 - (Y - y0)**2 - (Z - z0)**2)

data = gaussian(0.1, -0.1, -0.1) - gauss(-0.5, -0.5, -0.5) - gauss(0.5, 0.5, 0.5)

ipv.figure()
ipv.volshow(data)
ipv.view(-40)
ipv.show()

借助IPython提供的交互控件，可以使用下方的拖动条查看data的在xy方向上的二维数据切片。

In [8]:
from ipywidgets import interactive
import matplotlib.pyplot as plt

def slice_z(i):
    fig, ax = plt.subplots(figsize=(8, 8))
    im = ax.imshow(data[i,:,:], vmin=-1, vmax=1, cmap=plt.get_cmap('gist_rainbow'))
    ct = ax.contour(data[i,:,:])
    bar = plt.colorbar(im)
    plt.show()

interact_plot = interactive(slice_z, i=(0, 99))
interact_plot

interactive(children=(IntSlider(value=49, description='i', max=99), Output()), _dom_classes=('widget-interact'…

## 提示
---

寻找过渡态的一个常用算法是微动弹性带（[Nudged Elastic Band](https://theory.cm.utexas.edu/henkelman/pubs/jonsson98_385.pdf)）。它的核心思想是，将初始坐标和终态坐标用若干个中间态（例如11个）连接起来，然后让这些中间态沿着函数梯度的反方向移动（类似于小球在地形图中沿着山坡向下移动）；为了避免这些中间态都收敛到附近的局部极小（类似于小球都落入了盆地），相邻中间态之间用一根虚拟的弹簧连接，从而迫使相邻中间态有一定的间距。当这个小球弹簧链（微动弹性带）移动停止时，其所在位置就是所谓的最低能量路径（minimal energy path），其中间函数值最大的位置就是鞍点或者过渡态。

在迭代计算过程中，中间态的移动同时受*函数梯度*和*弹簧弹力*的调控。为了保持中间态的间距尽量不改变，以及虚拟弹簧不影响路径正确性，可以对*函数梯度*和*弹簧弹力*进行矢量分解。其中，*函数梯度*只保留垂直于路径的分量；*弹簧弹力*只保留沿着路径的分量。